# THIS IS WIP, IT DOES NOT WORK FOR NOW
# NEED TO SPECIFY THE MODEL

In [0]:
# imports
import os
from datetime import datetime

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from pyspark.sql import Window
from pyspark.sql import functions as F
from pyspark.sql import DataFrame as SparkDataFrame

In [0]:
VOLUME_ROOT_PATH = "/Volumes/cscie103_catalog/final_project/data"
# place where raw csvs land after download
VOLUME_TARGET_DIR = f"{VOLUME_ROOT_PATH}/raw"
# place where prepared data is written
VOLUME_PREPARED_DIR = f"{VOLUME_ROOT_PATH}/prepared"
# place where final data is written
VOLUME_FINAL_DIR = f"{VOLUME_ROOT_PATH}/final"

In [0]:
# ensure all directories exist
for dir in [VOLUME_TARGET_DIR, VOLUME_PREPARED_DIR, VOLUME_FINAL_DIR]:
    os.makedirs(dir, exist_ok=True)

In [0]:
# load the data from local volumes
filenames = {
    'holidays_events': 'holidays_events.csv',
    'oil': 'oil.csv',
    'sample_submission': 'sample_submission.csv',
    'stores': 'stores.csv',
    'test': 'test.csv',
    'train': 'train.csv',
    'transactions': 'transactions.csv'
}

# holidays_events_df = spark.read.csv(f"{VOLUME_TARGET_DIR}/{filenames.get('holidays_events')}", header=True, inferSchema=True)
# oil_df = spark.read.csv(f"{VOLUME_TARGET_DIR}/{filenames.get('oil')}", header=True, inferSchema=True)
stores_df = spark.read.csv(f"{VOLUME_TARGET_DIR}/{filenames.get('stores')}", header=True, inferSchema=True)
transactions_df = spark.read.csv(f"{VOLUME_TARGET_DIR}/{filenames.get('transactions')}", header=True, inferSchema=True)
train_df = spark.read.csv(f"{VOLUME_TARGET_DIR}/{filenames.get('train')}", header=True, inferSchema=True)

test_df = spark.read.csv(f"{VOLUME_TARGET_DIR}/{filenames.get('test')}", header=True, inferSchema=True)

In [0]:
# prepare: train_df
def smart_na_drop(df):
    before = df.count()
    df = df.dropna()
    after = df.count()
    print(f"dropped {before - after} rows")
    return df
train_df = smart_na_drop(train_df)

In [0]:
# prepare: transactions_df
transactions_df = smart_na_drop(transactions_df)

In [0]:
# prepare: stores_df
stores_df = smart_na_drop(stores_df)

In [0]:
# merge train_df & transactions_df by date & store_nbr
train_df = train_df.join(transactions_df, ['date', 'store_nbr'], how='left')
# merge metadata - stores_df
train_df = train_df.join(stores_df, ['store_nbr'], how='left')

# write to parquet
train_df.write.mode("overwrite").parquet(f"{VOLUME_PREPARED_DIR}/train_prepared")

In [0]:
display(train_df.dtypes)
display(train_df.count())

In [0]:
# write train_df in /raw
train_df.write.mode("overwrite").parquet(f"{VOLUME_TARGET_DIR}/train")

In [0]:
# reread from /raw
train_df = spark.read.parquet(f"{VOLUME_TARGET_DIR}/train")
train_df = train_df.toPandas()

# type casts
train_df['date'] = pd.to_datetime(train_df['date'])

train_df['store_nbr'] = train_df['store_nbr'].astype('int32')
# train_df['transactions'] = train_df['transactions'].astype('int32')
train_df['onpromotion'] = train_df['onpromotion'].astype('int32')
train_df['sales'] = train_df['sales'].astype('float64')

train_df['family'] = train_df['family'].astype('category')
train_df['city'] = train_df['city'].astype('category')
train_df['state'] = train_df['state'].astype('category')
train_df['type'] = train_df['type'].astype('category')
train_df['cluster'] = train_df['cluster'].astype('int32')

In [0]:
print(type(train_df))
print(train_df.columns.tolist())
# print types
for col in train_df.columns:
    print(col, train_df[col].dtype)

In [0]:
train_df['date']

In [0]:
# train_df preparation
# drop useless columns
train_df.drop(columns=['id'], axis=1, inplace=True)

# feature engineering for date
# add feature for previous day's sales
train_df['dayofweek'] = train_df['date'].dt.dayofweek # Note: Pandas dayofweek is 0 (Monday) to 6 (Sunday)
train_df['dayofmonth'] = train_df['date'].dt.day
train_df['month'] = train_df['date'].dt.month

# Sort the DataFrame first to ensure the .shift() operation works chronologically
train_df = train_df.sort_values(by=['store_nbr', 'family', 'date']).reset_index(drop=True)

# Define the groups (equivalent to partitionBy) and apply the shift (equivalent to lag(1))
train_df['sales_lag_1'] = train_df.groupby(['store_nbr', 'family'])['sales'].shift(1)

# 4. Handle rows where lag is null (first day for a store/family pair)
# This is equivalent to PySpark's .filter(F.col('sales_lag_1').isNotNull())
train_df = train_df.dropna(subset=['sales_lag_1'])

train_df = train_df.drop('date', axis=1)

In [0]:
print(train_df.columns.tolist())

In [0]:
%pip install tensorflow

In [0]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Embedding, Flatten, concatenate
from tensorflow.keras.models import Model
from sklearn.preprocessing import StandardScaler

# --- 0. Define Feature Groups (Based on your input) ---
CATEGORICAL_COLS = ['store_nbr', 'family', 'city', 'state', 'type', 'cluster', 
                    'dayofweek', 'dayofmonth', 'month']
CONTINUOUS_COLS = ['onpromotion', 'transactions', 'sales_lag_1']
TARGET_COL = 'sales'


# --- 1. Data Preparation and Chronological Split ---

# 1a. Sort Data Chronologically (CRITICAL for time series)
# Assuming 'date' is still available and is datetime type
# train_df = train_df.sort_values(by=['date', 'store_nbr', 'family']).reset_index(drop=True)


# 1b. Integer Encode Categorical Features
vocab_sizes = {}
for col in CATEGORICAL_COLS:
    # Convert to category type and get codes (0-indexed integers)
    train_df[col] = train_df[col].astype('category').cat.codes
    # Calculate vocabulary size (number of unique codes)
    vocab_sizes[col] = len(train_df[col].unique())


# 1c. Scale Continuous Features
scaler = StandardScaler()
train_df[CONTINUOUS_COLS] = scaler.fit_transform(train_df[CONTINUOUS_COLS])


# 1d. Define Split Point (80% Train, 20% Validation)
split_index = int(len(train_df) * 0.8)
train_set = train_df.iloc[:split_index]
val_set = train_df.iloc[split_index:]


# 1e. Prepare Input Arrays for Keras Model
def prepare_keras_inputs(df):
    """Generates the list of input arrays for the Keras model."""
    X_cat = [df[col].values for col in CATEGORICAL_COLS]
    X_cont = df[CONTINUOUS_COLS].values
    y = df[TARGET_COL].values
    
    X_inputs = X_cat + [X_cont]
    return X_inputs, y

X_train_inputs, y_train = prepare_keras_inputs(train_set)
X_val_inputs, y_val = prepare_keras_inputs(val_set)

print(f"Data Split Complete. Training samples: {len(y_train)}, Validation samples: {len(y_val)}")

In [0]:
# --- 2. FFNN Model Definition with Embedding Layers ---

# Function to determine optimal embedding dimension (common heuristic)
def get_embedding_dim(vocab_size):
    return min(50, (vocab_size // 2) + 1)

input_layers = []
cat_embeddings = []

# Process Categorical Inputs
for col in CATEGORICAL_COLS:
    vocab_size = vocab_sizes[col]
    emb_dim = get_embedding_dim(vocab_size)
    
    # 2a. Input Layer
    input_layer = Input(shape=(1,), name=f'{col}_input')
    input_layers.append(input_layer)

    # 2b. Embedding Layer (input_dim = vocab size + 1 for 0-indexing/padding)
    embedding = Embedding(
        input_dim=vocab_size + 1, 
        output_dim=emb_dim, 
        name=f'{col}_embed'
    )(input_layer)
    
    # 2c. Flatten the output
    flat = Flatten(name=f'{col}_flat')(embedding)
    cat_embeddings.append(flat)

# Process Continuous Inputs
N_CONT_FEATURES = len(CONTINUOUS_COLS)
cont_input = Input(shape=(N_CONT_FEATURES,), name='cont_input')
input_layers.append(cont_input) 

# Concatenate all features
merged = concatenate(cat_embeddings + [cont_input])

# Define the main FFNN layers
dense1 = Dense(128, activation='relu')(merged)
dense2 = Dense(64, activation='relu')(dense1)

# Output Layer for Sales Prediction (Regression)
output = Dense(1, activation='linear')(dense2)

# Create and Compile the final Model
model = Model(
    inputs=input_layers, 
    outputs=output
)

# Use Adam optimizer and Mean Squared Error (MSE) loss for regression
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

print("--- FFNN Model Architecture ---")
model.summary()

In [0]:
# --- 3. Model Training ---

print("\n--- Starting Model Training ---")

# Train the model using the chronologically split data
history = model.fit(
    X_train_inputs, 
    y_train, 
    validation_data=(X_val_inputs, y_val), 
    epochs=10, 
    batch_size=64, # Using a reasonable batch size
    verbose=1
)

print("\n--- Model Training Complete ---")